In [ ]:
from onediff.infer_compiler import oneflow_compile, oneflow_load_compiled
from onediff.schedulers import EulerDiscreteScheduler
from onediff.optimization import rewrite_self_attention
from diffusers import StableDiffusionPipeline
import torch
import os

In [12]:
compiled_graph_path = "/workspace/test"
prompt = "an icon of a cat"
model_id = "/workspace/sd-1_5-icons-172800_steps-4e_7"
height = 512
width = 512
steps = 50
warmup = 1
seed = 5

In [ ]:
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    scheduler=scheduler,
    use_auth_token=True,
    revision="fp16",
    variant="fp16",
    torch_dtype=torch.float16,
    safety_checker=None,
).to("cuda")

In [ ]:
pipe.unet = oneflow_load_compiled(pipe.unet, compiled_graph_path, device="cuda")

In [ ]:
torch.manual_seed(seed)

prompt = "an icon of a star"
images = pipe(
    prompt, height=height, width=width, num_inference_steps=steps
).images

for i, image in enumerate(images):
    image.save(f"{prompt}-of-{i}.png")